In [ ]:
import matplotlib.pyplot as plt
from selenium import webdriver
from bs4 import BeautifulSoup
import seaborn as sns
import pandas as pd
import time

In [ ]:
url = 'http://www.gamestar.de/news/'

start = time.time()
driver = webdriver.Firefox()
driver.get(url)

clicks = 1800
wait = 1.5
for _ in range(clicks):
    try:
        driver.find_element_by_xpath('//a[@title="Mehr anzeigen"]').click()
    except:
        pass
    time.sleep(wait)
ende = time.time()
print('Dauer für {} Klicks mit jeweils {}s Wartezeit: {}'.format(clicks, wait, ende-start))

In [ ]:
html = driver.page_source
driver.close()

In [ ]:
bs = BeautifulSoup(html, 'lxml')

In [ ]:
data = []
for elem in bs.find('span', {'class':'box-reload'}).findAll('div'):
    for div in elem.findAll('div'):
        if div.has_attr('class'):
            if div['class']==['col-xs-12', 'm-t-2', 'm-b-05']:
                day = div.get_text().split(' ')[-1]
            elif div['class']==['media', 'news-list', 'article-list', 'm-b-0']:
                time = div.find('span', {'class':'info news-info'}).get_text()
                text = div.find('a')['title']
                comments = div.find('span', {'class':'info-small'}).get_text()
                data.append([day+' '+time, text, int(comments)])
            else:
                pass
            
df = pd.DataFrame(data, columns=['Date', 'Title', 'Comments'])
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.set_index('Date')

def getMainTopic(x):
    return x.split(' - ')[0]

df['Topic'] = df['Title'].apply(lambda x: getMainTopic(x))
df.to_csv('build/Gamestar_News.csv')